In [1]:
install.packages("fpp3")
install.packages("urca")
library(fpp3)
library(ggplot2)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘quadprog’, ‘warp’, ‘BH’, ‘distributional’, ‘progressr’, ‘ggdist’, ‘slider’, ‘anytime’, ‘fable’, ‘fabletools’, ‘feasts’, ‘tsibble’, ‘tsibbledata’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Registered S3 method overwritten by 'tsibble':
  method               from 
  as_tibble.grouped_df dplyr

── Attaching packages ──────────────────────────────────────────── fpp3 1.0.0 ──

✔ tibble      3.2.1     ✔ tsibble     1.1.5
✔ dplyr       1.1.4     ✔ tsibbledata 0.4.1
✔ tidyr       1.3.1     ✔ feasts      0.3.2
✔ lubridate   1.9.3     ✔ fable       0.3.4
✔ ggplot2     3.5.1     ✔ fabletools  0.4.2

── Conflicts ───────────────────────────────────────────────── fpp3_conflicts ──
✖ lubridate::date()    masks base::date()
✖ dplyr::filter()      masks stats::filter()
✖ tsibble::intersect() masks base::intersect()
✖ tsibble::interv

### Read data

In [2]:
df <- read.csv('data.csv')
df$Timestamp <- as.POSIXct(df$Timestamp,format='%Y-%m-%d %H:%M:%S')
df <- as_tsibble(df,index=Timestamp)

In [ ]:
df |> colnames()

[1] "Month"                                          
 [2] "Hour"                                           
 [3] "DayType"                                        
 [4] "Workday"                                        
 [5] "DaylightSavings"                                
 [6] "Load"                                           
 [7] "Load_1"                                         
 [8] "Load_2"                                         
 [9] "Load_3"                                         
[10] "Load_4"                                         
[11] "Load_5"                                         
[12] "Load_6"                                         
[13] "Load_7"                                         
[14] "Load_8"                                         
[15] "Load_9"                                         
[16] "SolarGen"                                       
[17] "NetLoad"                                        
[18] "NetLoadLag"                                     
[19] "NetLoadLag2"                                    
[20] "NetLoadLag3"                                    
[21] "NetLoadLag24"                                   
[22] "NetLoadMax24"                                   
[23] "NetLoadMin24"                                   
[24] "Price"                                          
[25] "Outdoor.Drybulb.Temperature..C."                
[26] "Outdoor.Relative.Humidity...."                  
[27] "Diffuse.Solar.Radiation..W.m2."                 
[28] "Direct.Solar.Radiation..W.m2."                  
[29] "X6h.Prediction.Outdoor.Drybulb.Temperature..C." 
[30] "X12h.Prediction.Outdoor.Drybulb.Temperature..C."
[31] "X24h.Prediction.Outdoor.Drybulb.Temperature..C."
[32] "X6h.Prediction.Outdoor.Relative.Humidity...."   
[33] "X12h.Prediction.Outdoor.Relative.Humidity...."  
[34] "X24h.Prediction.Outdoor.Relative.Humidity...."  
[35] "X6h.Prediction.Diffuse.Solar.Radiation..W.m2."  
[36] "X12h.Prediction.Diffuse.Solar.Radiation..W.m2." 
[37] "X24h.Prediction.Diffuse.Solar.Radiation..W.m2." 
[38] "X6h.Prediction.Direct.Solar.Radiation..W.m2."   
[39] "X12h.Prediction.Direct.Solar.Radiation..W.m2."  
[40] "X24h.Prediction.Direct.Solar.Radiation..W.m2."  
[41] "Radiation..W.m2."                               
[42] "X6h.Prediction.Radiation..W.m2."                
[43] "X12h.Prediction.Radiation..W.m2."               
[44] "X24h.Prediction.Radiation..W.m2."               
[45] "TempLag"                                        
[46] "TempLag2"                                       
[47] "TempLag3"                                       
[48] "TempLag24"                                      
[49] "TempMean24"                                     
[50] "TempMean1W"                                     
[51] "Timestamp"

In [3]:
df <- df |> rename(Temp = Outdoor.Drybulb.Temperature..C.,
                    TempPred6h = X6h.Prediction.Outdoor.Drybulb.Temperature..C.,
                    TempPred12h = X12h.Prediction.Outdoor.Drybulb.Temperature..C.,
                    TempPred24h = X12h.Prediction.Outdoor.Drybulb.Temperature..C.,
                    RadDifPred6h = X6h.Prediction.Diffuse.Solar.Radiation..W.m2.,
                    RadDifPred12h = X12h.Prediction.Diffuse.Solar.Radiation..W.m2.,
                    RadDifPred24h = X24h.Prediction.Diffuse.Solar.Radiation..W.m2.,
                    RadDifPred24h = X24h.Prediction.Diffuse.Solar.Radiation..W.m2.,
                    RadDirPred6h = X6h.Prediction.Direct.Solar.Radiation..W.m2.,
                    Rad = Radiation..W.m2.,
                    RadPred6h = X6h.Prediction.Radiation..W.m2.,
                    RadPred12h = X12h.Prediction.Radiation..W.m2.,
                    RadPred24h = X24h.Prediction.Radiation..W.m2.,
                    Hum = Outdoor.Relative.Humidity....,
                    HumPred6h = X6h.Prediction.Outdoor.Relative.Humidity....,
                    HumPred12h = X12h.Prediction.Outdoor.Relative.Humidity....,
                    HumPred24h = X24h.Prediction.Outdoor.Relative.Humidity....,
                    Solar = SolarGen)
df$DayType <- as.factor(df$DayType)
df$Hour <- as.factor(df$Hour)
df$Month <- as.factor(df$Month)
df$Holiday <- (df$DayType == "Holiday")
df$Sun <- (df$DayType == "Sun")
df$Sat <- (df$DayType == "Sat")

In [4]:
train <- df[1:(24*365*2),]
val <- df[(24*365*2+1):(24*365*3),]
test <- df[(24*365*3+1):nrow(df),]

In [56]:
snaive_innov <- readRDS("SNAIVE.rds") |> refit(test) |> residuals()
sarima_innov <- readRDS("SARIMA.rds") |> refit(test) |> residuals()
ets_innov <- readRDS("ETS.rds") |> refit(test) |> residuals()
reg_innov <- readRDS("REG.rds") |> refit(test) |> residuals()
nnarx_innov <- readRDS("NNARX.rds") |> refit(test) |> residuals()

.model,Timestamp,.resid
<chr>,<dttm>,<dbl>
model,2019-01-01 01:00:00,NA
model,2019-01-01 02:00:00,1.42034355
model,2019-01-01 03:00:00,2.01900463
model,2019-01-01 04:00:00,1.17641558
model,2019-01-01 05:00:00,0.76268925
model,2019-01-01 06:00:00,0.61055505
model,2019-01-01 07:00:00,-1.22661046
model,2019-01-01 08:00:00,-1.49735352
model,2019-01-01 09:00:00,-1.58141990
